In [1]:
## 通用函數：提供 source template file， target: 套表完成的路徑。
def genOneCode(source, target, _dictParas):
    _source = "template/" + source
    fileName = target
    # print("source:", _source)
    # print("target:", fileName)

    tmp = codecs.open(_source, mode="r",encoding="utf-8", buffering=-1).read()
    t = Template(tmp).substitute(_dictParas)
    os.makedirs(os.path.dirname(fileName), exist_ok=True)
    f = codecs.open(fileName, mode="w", encoding="utf-8", errors="strict")
    f.write(t)
    f.close()

import pyodbc
#寫入到資料庫中.....
def ExecuteMSSQL(sql):
    server = '20.121.2.79'
    database = 'newcti'
    username = 'agddev_db_admin'
    password = 'pDataMIMA99*'
    driver= '{ODBC Driver 13 for SQL Server}'
    cnxn = pyodbc.connect('DRIVER='+driver+';PORT=1433;SERVER='+server+';PORT=1443;DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    
    cursor.execute(sql)
    row = cursor.fetchone()     #查詢
    # cursor.commit()            #執行
    # rs = []
    # while row: 
    #     # print(row[0])
    #     row = cursor.fetchone()
    #     rs.append(row)
    return row

rss = ExecuteMSSQL("select top 1 *, convert(varchar(20), getdate(),120) as cc from agdset.tbcode where codeType = 'TxnItemCode'")
print(rss)


def fmtColumns(r):
    col, colDscr, typ = r["Column"], r["ColDscr"], r["Type"]
    tmp = Template("""$col = "$col",\n""")
    return tmp.substitute(col = col, typ = typ)

def fmtConvertDT(r):
    col, typ = r["Column"], r["Type"]
    tmp = Template("""convert(varchar(20), $col, 120) as '$col'""") if (typ in ["datetime2","datetime","date","time"]) else Template("$col")
    return tmp.substitute(col = col)

def fmtJSColumn(r):
    col = r["Column"]
    return col[0:1].lower() + col[1:]

(1, 'TxnItemCode', 'TIC01', '掛失金融卡/簽帳金融卡/勞動保障卡', '', '', 0, True, datetime.datetime(2022, 2, 2, 12, 0), 'sys', '系統', datetime.datetime(2022, 2, 2, 12, 0), 'sys', '系統', '2022-10-27 12:24:57')


C:\Users\user\AppData\Local\Temp\ipykernel_7008\1046284914.py:37: DeprecationWarning: PyUnicode_FromUnicode(NULL, size) is deprecated; use PyUnicode_New() instead
  print(rss)


In [3]:
import pandas as pd
import numpy as np

import comm as cmm
import time
import codecs
import math
from string import Template

_ExcelFile = "refFiles/玉山AGD專案DB_Schema_20221024.xlsx"
pd.options.mode.chained_assignment = None  # default='warn'
class MyTemplate(Template):
    delimiter = "//$"           #自己定義的替換字元符號

df = pd.read_excel(_ExcelFile, sheet_name="DB Schema",converters={'maxLen':str }).replace(np.nan, '')
df["schemaTableName"] = df["Schema"] + ".tb" + df["TableName"]

tbs         = df[df["TableName"] != "-"]["TableName"].unique()
schemaTbs   = df[df["TableName"] != "-"]["schemaTableName"].unique()

print("tables:", tbs)
# print("tables:", schemaTbs)

df["Column"]        = df.apply(fmtJSColumn, axis = 1)
df["Column2"]       = df.apply(fmtConvertDT, axis = 1)
df["Columns"]       = df["Column2"] + ","
df["InsertColumns"] = df.apply(fmtColumns, axis = 1)


i = 0
for tb in tbs:
    
    if (tb in ['TxnLogNoSale'
        # 'CustAddress', 'CustEmail', 'UserPhoto','UserWordPad', 'UserNotifyLog', 'UserHist', 'RoleHist', 'UserRoleHist','RoleFuncHist', 'ReasonHist', 'CodeHist', 'MarketingOpenHist', 'Contact','ContactCustomer', 'ContactCustomerReason', 'Recording', 
#  'RecordingFiles',
#  'RecordingEvaluation', 'RecordingEvaluationAnswer', 'NotificationLog',
#  'MessageLog', 
#  'TxnLog', 
 #'MarketingLog', 'TxnLogLoss','TxnLogAccident', 
#  'TxnLogNoSale'
#  'AuthCallLog', 'CustMemo', 'CustMemoLog', 'CustQueryLog',
#  'LoginOutLog', 'EvaluationForm', 'BusinessDay', 'AutoNextNum', 'HubClients',
#  'HubMessages'
 ]):
        print(tb)
        schemaTableName     = schemaTbs[i]                      ## 完整schema table name
        escape4C = ["seqNo", "createDT", "updateDT", "updater", "updaterName"]   ## 排除掉4C

        _columnsList        = df[(df["TableName"] == tb) & (~df["Column"].isin(escape4C))]["Column"].tolist()
        _typeList           = df[(df["TableName"] == tb) & (~df["Column"].isin(escape4C))]["Type"].tolist()
        _queryList          = df[(df["TableName"] == tb) & (~df["Column"].isin(escape4C))]["Query"].tolist()
        _columns          = "".join(df[(df["TableName"] == tb) & (~df["Column"].isin(escape4C))]["Columns"].tolist())[:-1]
        insertSql           = Template("select top 1 $_columns from $schemaTableName order by 1 asc; ").substitute(_columns = _columns, schemaTableName = schemaTableName)
        updateSql           = Template("select top 1 $_columns from $schemaTableName order by 1 desc;").substitute(_columns = _columns, schemaTableName = schemaTableName)


        print(insertSql)
        rss = ExecuteMSSQL(insertSql)
        urss = ExecuteMSSQL(updateSql)
        print(schemaTableName, rss, urss)

        j, pt_ColumnsInsert, pt_queryParas, pt_ColumnsUpdate = 0, "", "", ""

        for c in _columnsList:
            col     = _columnsList[j]
            type    = _typeList[j]
            isQuery = _queryList[j]  #是否為query條件?
            tmp     = "\t\t\t\t$col = $val,\n" if (type in ["int", "tinyint", "smallint", "float","decimal","numeric","bit"]) else """\t\t\t\t$col = "$val",\n"""
            pt_ColumnsInsert += Template(tmp).substitute(col = c, val = rss[j])
            pt_ColumnsUpdate += "" if (col in ["creator", "creatorName"]) else Template(tmp).substitute(col = c, val = urss[j])
            pt_queryParas += "" if (isQuery == 0) else "\t\t"+Template(tmp).substitute(col = c, val = rss[j])
            j += 1

        dictParas = dict(pt_TableName = tb, pt_Service="Service", pt_Test = "Test", pt_TestFailed = "TestFailed", pt_QueryRequest = "QueryRequest", pt_Insert = "Insert", pt_Update = "Update", pt_Delete = "Delete", pt_InsertRequest="InsertRequest", pt_UpdateRequest = "UpdateRequest", pt_Get = "Get", pt_Response ="Response", pt_Request ="Request", pt_Query = "Query", pt_ColumnsInsert = pt_ColumnsInsert[:-1], pt_ColumnsUpdate = pt_ColumnsUpdate[:-1], pt_queryParas = pt_queryParas[:-1])

        ## 開始套表...
        genOneCode("WebApiTest/TestPcPhoneService.cs", f"WebApiTest/{tb}Test/Test{tb}Service.cs", dictParas)
    i += 1



tables: ['User' 'Employee' 'Group' 'Role' 'Func' 'UserRole' 'RoleFunc' 'PcPhone'
 'AuxCode' 'IvrCode' 'Code' 'PhoneBook' 'Reason' 'ReasonIvr' 'ReasonTxn'
 'CipCode' 'CipImportCode' 'UserShift' 'UserOvertime'
 'NotificationTemplate' 'MessageSheet' 'Message' 'MessageTemplate' 'Txn'
 'DebitCardCode' 'AccidentFormTemplate' 'AccidentScript' 'ZipCity'
 'ZipRoad' 'Marketing' 'MarketingActionCode' 'MealScript' 'SysConfig'
 'Cust' 'CustKYC' 'CustPhone' 'CustAddress' 'CustEmail' 'UserPhoto'
 'UserWordPad' 'UserNotifyLog' 'UserHist' 'RoleHist' 'UserRoleHist'
 'RoleFuncHist' 'ReasonHist' 'CodeHist' 'MarketingOpenHist' 'Contact'
 'ContactCustomer' 'ContactCustomerReason' 'Recording' 'RecordingFiles'
 'RecordingEvaluation ' 'RecordingEvaluationAnswer ' 'NotificationLog'
 'MessageLog' 'TxnLog' 'MarketingLog' 'TxnLogLoss' 'TxnLogNoSale'
 'TxnLogAccident' 'AuthCallLog' 'CustMemo' 'CustMemoLog' 'CustQueryLog'
 'LoginOutLog' 'EvaluationForm' 'BusinessDay' 'AutoNextNum' 'HubClients'
 'HubMessages']
TxnLog

C:\Users\user\AppData\Local\Temp\ipykernel_7008\2561310481.py:60: DeprecationWarning: PyUnicode_FromUnicode(NULL, size) is deprecated; use PyUnicode_New() instead
  print(schemaTableName, rss, urss)
